# IEOR 4501 Final Project: Understanding Hired Rides in NYC

#### Contributors: Joy Ren(jr4154), Yiwen Qian(yq2346)

## Project Setup

In [3]:
# all import statements needed for the project

import math
from math import sin, cos, sqrt, atan2, radians
import os
import warnings
import re
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd
warnings.filterwarnings('ignore')

In [4]:
# any constants need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_DATA = "uber_rides_sample.csv"
WEATHER_CSV_DIR = "weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [5]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing
In Part 1, we downloaded Parquet files, cleaning and filtering for the relevant data, filling in missing data, and generating samples of these raw datasets.
Our processing stages can be illustrated as:
* 1. Load the Taxi Zones by x and y coordinates.
* 2. Calculate distance using latitude and longtitude.
* 3. Use `bs4` and `requests` module to parse html, then we can process Yellow Taxi data of NYC.
* 4. Processing Uber Data.
* 5. Processing Weather Data.
* 6. Save the cleaned data for further analyis. 

### Load Taxi Zones

In [6]:
def load_taxi_zones(shapefile):
    return gpd.read_file(shapefile)     #read shapefile

In [7]:
load_taxi_zones(TAXI_ZONES_SHAPEFILE)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


In [8]:
def lookup_coords_for_taxi_zone_id(zone_loc_id):
    zones= load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    centroid= zones[zones['LocationID'] == zone_loc_id].centroid.values[0]
    return (centroid.y, centroid.x)        # return the centroid coordinates

### Calculate distance

In [9]:
def calculate_distance_with_coords(from_coord, to_coord):
    ## "from_coord": columns 'pickup_latitude' and 'pickup_longitude' from datarame;
    ## "to_coord":  columns 'dropoff_latitude' and 'dropoff_longitude' from datarame;
    
    R = 6373.0  # approximate radius of earth in km    
    lat1, lon1 = from_coord
    lat2, lon2 = to_coord

    dlat = radians(lat2-lat1)
    dlon = radians(lon2-lon1)
    
    ## Calculate the distance between two coordinates in kilometers using the Haversine formula
    a = sin(dlat/2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * (sin(dlon/2) ** 2 )
    coord_distance = 2 * R * atan2(math.sqrt(a), sqrt(1-a))
    
    return coord_distance

In [10]:
def calculate_distance_with_zones(from_zone, to_zone):
    ##Calculate the distance between two taxi zones in kilometers using the centroid of each zone
    
    from_zone_coord = lookup_coords_for_taxi_zone_id(from_zone)
    to_zone_coord = lookup_coords_for_taxi_zone_id(to_zone)   # get two coordinates from two zone ids
    
    return calculate_distance_with_coords(from_zone_coord, to_zone_coord)

In [11]:
def add_distance_column(dataframe):               
    distance=[]
    for index, row in dataframe.iterrows():
        from_coord=(row['pickup_latitude'],row['pickup_longitude'])
        to_coord=(row['dropoff_latitude'],row['dropoff_longitude'])
        distance.append(calculate_distance_with_coords(from_coord,to_coord))  
    dataframe= pd.concat([dataframe,pd.DataFrame(distance,columns=['distance'])], axis=1)
    return dataframe

### Process Taxi Data
Here, We want to obtain the Yellow Taxi Data January 2009 through June 2015, then do the cleaning and combining to dataframes.

In [12]:
def get_all_urls_from_taxi_page(taxi_page):
    # Get the HTML content of the page
    res = requests.get(taxi_page)
    soup = bs4.BeautifulSoup(res.content, 'html.parser')
    # Find all <a> tags with href attribute
    urls = [a['href'] for a in soup.find_all('a', href=re.compile(".*(2009|201[0-4]|2015-0[1-6]).*"))]
    return urls

In [13]:
all_urls=get_all_urls_from_taxi_page(TAXI_URL)
all_urls

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2015-04.parquet',
 'https:

In [14]:
def filter_taxi_parquet_urls(all_urls):
    """
    Given a list of URLs, filters out the URLs that do not contain yellow taxi data parquet files.
    """
    parquet_urls = []
    for url in all_urls:
        if 'yellow_tripdata' in url:
            parquet_urls.append(url)
    return parquet_urls

In [17]:
def get_and_clean_month(url):
    """
    clean the data, and return a pandas DataFrame.
    """          

    df = pd.read_parquet(url)
    df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'fare_amount']]
    df = df.dropna()


    
    zone=load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    zone = zone.to_crs(4326)
    zone['latitude'] = zone.geometry.centroid.y
    zone['longitude'] = zone.geometry.centroid.x
    

    # remove unnecessary columns 
    # create mapping methods for latitude & longitude
    # aim to create lat & longitude which do not exist in some taxi data
    zone = zone[['LocationID', 'longitude', 'latitude', 'zone', 'borough']]
    lat_map = dict(zip(zone['LocationID'], zone['latitude']))
    lon_map = dict(zip(zone['LocationID'], zone['longitude']))
    
    
    # process dataframe which only has LocationIDs 
    # match the LocationID in 'df' with dataframe 'g'
    # add 4 new columns to 'df' after matching
    if 'DOLocationID' in df:
        df['pickup_latitude']  = df['PULocationID'].map(lat_map)
        df['pickup_longitude'] = df['PULocationID'].map(lon_map)
        df['dropoff_latitude'] = df['DOLocationID'].map(lat_map)
        df['dropoff_longitude']= df['DOLocationID'].map(lon_map)
    
    # normalizing column names
    df = df.rename(columns={'tpep_pickup_datetime':'pickup_datetime', 
                            'Trip_Pickup_DateTime':'pickup_datetime', 
                            'tpep_dropoff_datetime':'dropoff_datetime',
                            'Fare_Amt':'fare_amount'         
                            })
        
    
    df = df[(df['fare_amount'] >0)]     
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
    df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_day'] = df['pickup_datetime'].dt.day
    df['pickup_weekday'] = df['pickup_datetime'].dt.weekday
    df['pickup_month'] = df['pickup_datetime'].dt.month
    df['pickup_year'] = df['pickup_datetime'].dt.year
    
    
    #df['distance_km'] = df.apply(lambda row: calculate_distance_with_zones(row['PULocationID'], row['DOLocationID']), axis=1)
    return df

In [18]:
get_and_clean_month("yellow taxi/yellow_tripdata_2014-02.parquet")

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,fare_amount,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,pickup_hour,pickup_day,pickup_weekday,pickup_month,pickup_year
0,2014-02-01 00:30:05,2014-02-01 00:30:17,140,140,2.5,40.765484,-73.954739,40.765484,-73.954739,0,1,5,2,2014
1,2014-02-01 00:16:52,2014-02-01 00:18:22,186,186,3.0,40.748497,-73.992438,40.748497,-73.992438,0,1,5,2,2014
2,2014-02-01 00:05:03,2014-02-01 00:15:44,107,249,9.5,40.736824,-73.984052,40.734576,-74.002875,0,1,5,2,2014
3,2014-02-01 00:17:19,2014-02-01 00:20:15,158,90,4.5,40.735035,-74.008984,40.742279,-73.996971,0,1,5,2,2014
4,2014-02-01 00:24:08,2014-02-01 00:37:07,234,246,11.5,40.740337,-73.990458,40.753309,-74.004015,0,1,5,2,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13063789,2014-02-28 23:15:00,2014-02-28 23:38:00,231,61,21.0,40.717773,-74.007880,40.674469,-73.939287,23,28,4,2,2014
13063790,2014-02-28 23:53:00,2014-03-01 00:12:00,225,177,14.0,40.688168,-73.931888,40.676644,-73.913632,23,28,4,2,2014
13063791,2014-02-28 23:12:00,2014-02-28 23:29:00,181,71,16.0,40.670374,-73.981414,40.644288,-73.937966,23,28,4,2,2014
13063792,2014-02-28 23:20:56,2014-02-28 23:55:20,92,92,27.5,40.761102,-73.828859,40.761102,-73.828859,23,28,4,2,2014


In [19]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
    
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        add_distance_column(dataframe)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [20]:
def get_taxi_data():
    # download file if not exist
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = filter_taxi_parquet_urls(all_urls)
    for url in all_parquet_urls:
        response = requests.get(url, stream=True)
        file_name = url.split("/")[-1]
        local_file_path = os.path.join("yellow taxi", file_name)
        if not os.path.exists(local_file_path):
            with open(local_file_path, "wb") as file:
                file.write(response.content)
    
    
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

### Processing Uber Data

In [21]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.dropna(how='any')      #remove missing values
    df = df.drop(columns=['Unnamed: 0','key','passenger_count'])      #remove unnecessary columns
    df = df.loc[df.pickup_longitude.between(-74.242330,-73.717047) & df.dropoff_longitude.between(-74.242330,-73.717047) 
         & df.pickup_latitude.between(40.560445,40.908524)& df.dropoff_latitude.between(40.560445,40.908524)]
          # remove locations out of range
    df.reset_index(drop=True, inplace=True)    #reset index
    return df

In [22]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    uber_dataframe = add_distance_column(uber_dataframe)
    uber_dataframe["pickup_datetime"] = pd.to_datetime(uber_dataframe['pickup_datetime'])
    uber_dataframe["year"] = uber_dataframe["pickup_datetime"].dt.year
    uber_dataframe["month"] = uber_dataframe["pickup_datetime"].dt.month
    uber_dataframe["day"] = uber_dataframe["pickup_datetime"].dt.day
    uber_dataframe["hour"] = uber_dataframe["pickup_datetime"].dt.hour
    uber_dataframe["dayofweek"] = uber_dataframe["pickup_datetime"].dt.dayofweek
    return uber_dataframe

In [23]:
uber_data = get_uber_data()

In [24]:
uber_data.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance,year,month,day,hour,dayofweek
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1.683851,2015,5,7,19,3
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,2.458361,2009,7,17,20,4
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,5.037958,2009,8,24,21,0
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,1.662205,2009,6,26,8,4
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,4.476855,2014,8,28,17,3


### Processing Weather Data

In [25]:
def get_all_weather_csvs(directory):
    csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]
    return csv_files

In [26]:
# get all the "weather" csv files from 2009-2015
get_all_weather_csvs(WEATHER_CSV_DIR)

['2012_weather.csv',
 '2011_weather.csv',
 '2014_weather.csv',
 '2013_weather.csv',
 '2009_weather.csv',
 '2015_weather.csv',
 '2010_weather.csv']

In [27]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    
    # remove columns that have all NaN values
    df=df.dropna(axis=1, how="all")
    
    # for "hourly wind speed", fill missing value (calculate hourly-data from daily-data)
    if df['HourlyWindSpeed'] is 'NaN':
        df['HourlyWindSpeed'] = df['DailyAverageWindSpeed']/24
        df['HourlyWindSpeed'].apply(lambda x: round(x,2))
        
    # only keep necessary columns
    keep = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']
    
    #in "hourly precipation" column, replace "T" with value 0, replace NaN with 0 since existing value 
    # for the column is small
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace({pd.np.nan: 0, 'T': 0})
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].apply(  lambda x: x.replace('s', '') 
                                                                if isinstance(x, str) else x)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].astype(float)
    
    df1 = pd.DataFrame()
    for i in keep:
        col1=df.loc[:,df.columns.str.contains(i)]
        df1=pd.concat([df1, col1], axis=1)
    hourly_clean= df1.dropna()
    hourly_clean.reset_index(drop=True, inplace=True)
    
    # Add columns of year, month, day and hour
    hourly_clean["DATE"] = pd.to_datetime(hourly_clean['DATE'])
    hourly_clean["YEAR"] = hourly_clean["DATE"].dt.year
    hourly_clean["MONTH"] = hourly_clean["DATE"].dt.month
    hourly_clean["DAY"] = hourly_clean["DATE"].dt.day
    hourly_clean["HOUR"] = hourly_clean["DATE"].dt.hour
    hourly_clean['HourlyPrecipitation'] = hourly_clean['HourlyPrecipitation'].round(2)
    hourly_clean.dropna(inplace=True)
    return hourly_clean

In [28]:
clean_month_weather_data_hourly('weather/2012_weather.csv')

,DATE,HourlyPrecipitation,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR
0,2012-01-01 00:51:00,0.0,6.0,2012,1,1,0
1,2012-01-01 01:51:00,0.0,7.0,2012,1,1,1
2,2012-01-01 02:51:00,0.0,6.0,2012,1,1,2
3,2012-01-01 03:51:00,0.0,5.0,2012,1,1,3
4,2012-01-01 04:51:00,0.0,0.0,2012,1,1,4
...,...,...,...,...,...,...,...
10203,2012-12-31 18:51:00,0.0,9.0,2012,12,31,18
10204,2012-12-31 19:51:00,0.0,9.0,2012,12,31,19
10205,2012-12-31 20:51:00,0.0,8.0,2012,12,31,20
10206,2012-12-31 21:51:00,0.0,7.0,2012,12,31,21


In [29]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file)
    
    #remove columns having all NaN values
    df = df.dropna(axis=1, how="all")
    
    #in "hourly precipation" column, replace "T" with value 0, replace NaN with 0 since existing value 
    # for the column is small
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace({pd.np.nan: 0, 'T': 0})
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].apply(  lambda x: x.replace('s', '') 
                                                                if isinstance(x, str) else x)
    df['HourlyPrecipitation'] = df['HourlyPrecipitation'].astype(float)
    
    df['DATE'] = pd.to_datetime(df['DATE']) 
    df['DATE'] = df['DATE'].apply(lambda x: x.date())
    
    # Group by same date, calculate the mean,use that number as daily data
    Daily_w = df.groupby('DATE')['HourlyWindSpeed'].mean()
    Daily_p = df.groupby('DATE')['HourlyPrecipitation'].mean()
    #df1 = df.drop_duplicates(subset=['DATE'])
    df = df[['DATE']]
    df = pd.merge(df, Daily_w, on='DATE')
    df= pd.merge(df, Daily_p, on='DATE')
    
    df.rename(columns={'HourlyWindSpeed':'DailyWindSpeed','HourlyPrecipitation':'DailyPrecipitation'},inplace=True)
    
    df['DailyWindSpeed'] = df['DailyWindSpeed'].round(2)
    df['DailyPrecipitation'] = df['DailyPrecipitation'].round(2)
    
    # Add columns of year, month, day and hour
    df["DATE"] = pd.to_datetime(df['DATE'])
    df["YEAR"] = df["DATE"].dt.year
    df["MONTH"] = df["DATE"].dt.month
    df["DAY"] = df["DATE"].dt.day
    df["HOUR"] = df["DATE"].dt.hour
    
    return df

In [30]:
clean_month_weather_data_daily('weather/2012_weather.csv')

,DATE,DailyWindSpeed,DailyPrecipitation,YEAR,MONTH,DAY,HOUR
0,2012-01-01,5.87,0.0,2012,1,1,0
1,2012-01-01,5.87,0.0,2012,1,1,0
2,2012-01-01,5.87,0.0,2012,1,1,0
3,2012-01-01,5.87,0.0,2012,1,1,0
4,2012-01-01,5.87,0.0,2012,1,1,0
...,...,...,...,...,...,...,...
10687,2012-12-31,7.52,0.0,2012,12,31,0
10688,2012-12-31,7.52,0.0,2012,12,31,0
10689,2012-12-31,7.52,0.0,2012,12,31,0
10690,2012-12-31,7.52,0.0,2012,12,31,0


In [31]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(WEATHER_CSV_DIR+"/"+csv_file)
        daily_dataframe = clean_month_weather_data_daily(WEATHER_CSV_DIR+"/"+csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [32]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [33]:
hourly_weather_data.head()

,DATE,HourlyPrecipitation,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR
0,2012-01-01 00:51:00,0.0,6.0,2012,1,1,0
1,2012-01-01 01:51:00,0.0,7.0,2012,1,1,1
2,2012-01-01 02:51:00,0.0,6.0,2012,1,1,2
3,2012-01-01 03:51:00,0.0,5.0,2012,1,1,3
4,2012-01-01 04:51:00,0.0,0.0,2012,1,1,4


In [34]:
daily_weather_data.head()

,DATE,DailyWindSpeed,DailyPrecipitation,YEAR,MONTH,DAY,HOUR
0,2012-01-01,5.87,0.0,2012,1,1,0
1,2012-01-01,5.87,0.0,2012,1,1,0
2,2012-01-01,5.87,0.0,2012,1,1,0
3,2012-01-01,5.87,0.0,2012,1,1,0
4,2012-01-01,5.87,0.0,2012,1,1,0


## Part 2: Storing Cleaned Data

In [35]:
engine = db.create_engine(DATABASE_URL)

In [39]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    DailyWindSpeed FLOAT,
    DailyPrecipitation FLOAT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY,
    pickup_datetime DATE,
    dropoff_datetime DATE,
    PULocationID INTEGER,
    DOLocationID INTEGER,
    fare_amount FLOAT,
    pickup_latitude FLOAT,
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT,
    distance FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    fare_amount FLOAT,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER,
    dayofweek INTEGER
);
"""

In [40]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [41]:
# create the tables with the schema files
with engine.connect() as connection:
    afile = open(DATABASE_SCHEMA_FILE, 'r')
    sqlFile = afile.read()
    afile.close()

    sqlQuery = sqlFile.split(';')
    for query in sqlQuery:
        connection.execute(query)

### Add Data to Database

In [42]:
def write_dataframes_to_table(table_to_df_dict):
    for table, df in table_to_df_dict.items():
        df.to_sql(table, engine,if_exists='append', index=False)

In [46]:
map_table_name_to_dataframe = {
    #"taxi_trips": taxi_data,                 taxi数据没好
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [47]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(QUERY_DIRECTORY + outfile, "w") as f:
        f.write(query)

### Query 1:For 01-2009 through 06-2015, show the popularity of Yellow Taxi rides for each hour of the day. The query result should have 24 bins, one for each hour, descending.


In [1]:
QUERY_1_FILENAME = ""

QUERY_1 = """

"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2: For the same time frame, show the popularity of Uber rides for each day of the week.The result should have 7 bins, one for each day, descending.

### Query 3: What is the 95% percentile of distance traveled for all hired trips during July 2013? The result should be a float. It’s okay if it’s a single float within a list and/or tuple.


### Query 4:  What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day? 

#### The result should be a list of 10 tuples. Each tuple should have three items: a date, an integer for the number of rides of that date, and a float for the average distance of that date. The list of tuples should be sorted by total number of rides, descending.

### Query 5: Which 10 days in 2014 were the windiest on average, and how many hired trips were made on those days? 

#### The result should be a list of 10 tuples. Each tuple should have three items: a date, a float for the average wind speed of that day, and the number of hired trips for that day. The list of tuples should be sorted by the average wind speed, descending.

### Query 6: During Hurricane Sandy in NYC (Oct 29-30, 2012), plus the week leading up and the week after, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed? 


#### The result should be a list of roughly 384 tuples, where each tuple is an entry for every single hour of the given date range, even if no rides were taken, no precipitation was measured, or there was no wind. Each tuple should have four items: a string for the date and hour, an int for the number of hired rides in that hour, the float for the total precipitation for that hour, and a float for the average wind speed for that hour. The list of tuples should be ordered by date+hour, ascending.


## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)